In [64]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

input_path = '../data/stock_data/'
output_path = '../data/output/'

def table1(input_path,holding_days=5):
    # 读取数据
    # stock = 'sh600000'
    # set path to 'data/balabala' if you run Python file
    # input_path = '../data/stock_data/sh600000.csv'
    # df_input = pd.read_csv(input_path, skiprows=1, encoding='GB18030')
    df_input = pd.read_csv(input_path, skiprows=1, encoding='GBK')


    stock_trade_record = pd.DataFrame(columns=['股票代码', '股票名称', '持有期限', '买入日期','卖出日期', '买入价格', '卖出价格','收益率', '浮盈', '浮亏'])


    # 数据初始化
    peak_value = df_input['总市值'][0]
    stock_holding = False  # 判断是否持有股票
    return_rate = 1.0  # 收益率
    max_return_rate = 1.0  # 浮亏
    min_return_rate = 1.0  # 浮亏
    win = 0  # 亏利次数
    loss = 0  # 亏损次数
    trade_times = 0  # 交易次数
    # holding_days = 5  # 持有期限

    # 遍历数据，找到股票价值达到新高的时刻
    for i in range(len(df_input)):
        if stock_holding == True:
            date = datetime.datetime.strptime(df_input['交易日期'][i], '%Y-%m-%d')
            # 单日涨跌幅
            change = (df_input['收盘价'][i]/df_input['前收盘价'][i]) - 1

            # 累计收率率
            return_rate *= (1 + change)
            # 最大浮亏
            if return_rate > max_return_rate:
                max_return_rate = return_rate
            # 最大浮亏
            if return_rate < min_return_rate:
                min_return_rate = return_rate

            # 判断股票的持有期限，此处为5天
            # 持有时间超过期限时，计算指标数据
            if (date - buy_date).days >= holding_days:
                stock_holding = False
                # 当股票涨停或跌停时，无法对股票进行操作
                if df_input['开盘价'][i] != df_input['最高价'][i] and \
                        df_input['最高价'][i] != df_input['最低价'][i] and \
                        df_input['开盘价'][i] != df_input['最低价'][i]:
                    sell_date = datetime.datetime.strptime(df_input['交易日期'][i], '%Y-%m-%d')
                    sell_value = df_input['开盘价'][i]
                    if return_rate >= 1:
                        win += 1
                    else:
                        loss += 1
                    stock_trade_record.loc[trade_times] = [df_input['股票代码'][i], df_input['股票名称'][i], holding_days,
                                                        buy_date, sell_date, buy_value, sell_value, return_rate,
                                                        max_return_rate, min_return_rate]
                    trade_times += 1
                    return_rate = 1.0
                    max_return_rate = 1.0
                    min_return_rate = 1.0

        # 股票市值达到新高时，认为股票价值达到新高，买入股票并持有
        if df_input['总市值'][i] > peak_value and stock_holding == False:
            peak_value = df_input['总市值'][i]
            stock_holding = True
            buy_date = datetime.datetime.strptime(df_input['交易日期'][i], '%Y-%m-%d')
            buy_value = df_input['开盘价'][i]

    return stock_trade_record

def table2(table1):
    s = table1
    t2 = s[['股票代码', '股票名称', '持有期限']].loc[0]
    t2['胜率'] = (s['收益率']>1).sum() / len(s)
    t2['平均收益'] = s['收益率'].mean()
    t2['最大收益'] = s['收益率'].max()
    t2['最小收益'] = s['收益率'].min()
    t2['平均浮盈'] = s['浮盈'].mean()
    t2['最大浮盈'] = s['浮盈'].max()
    t2['最小浮盈'] = s['浮盈'].min()
    t2['平均浮亏'] = s['浮亏'].mean()
    t2['最大浮亏'] = s['浮亏'].max()
    t2['最小浮亏'] = s['浮亏'].min()

    return t2


file_list = os.listdir(input_path)
csv_list = []
df_output = pd.DataFrame(columns=['股票代码', '股票名称', '持有期限', '胜率', '平均收益', '最大收益', '最小收益', '平均浮盈', '最大浮盈',
        '最小浮盈', '平均浮亏', '最大浮亏', '最小浮亏'])

for holding_days in [5,10,15,20,25,30,35,40]:

    for names in file_list:
        if names.endswith(".csv"):
            csv_list.append(names)

    for file_name in csv_list[:100]:
        # print(file_name)
        file_path = os.path.join(input_path,file_name)
        t1 = table1(file_path,holding_days=holding_days)
        # print(t1)
        if len(t1) > 0:
            t2 = table2(t1)
        else:
            continue
        # print(t2)
        df_output = df_output.append(t2)
# df_output.to_excel(output_path + 'result'+ str(holding_days) +'.xlsx',index=False)


In [65]:
df_group_stock = df_output.groupby(['股票代码'])
for name, group in df_group_stock:
    group.to_excel(output_path + 'groupByStock/' + str(name) + '.xlsx',index=False)
df_group_duration = df_output.groupby(['持有期限'])
for name, group in df_group_duration:
    group.to_excel(output_path + 'groupByDuration/' + str(name) + '.xlsx',index=False)

,股票代码,股票名称,持有期限,胜率,平均收益,最大收益,最小收益,平均浮盈,最大浮盈,最小浮盈,平均浮亏,最大浮亏,最小浮亏
0,sh600000,浦发银行,5,0.452381,0.996992,1.168845,0.798088,1.029186,1.168845,1.0,0.959237,1.0,0.798088


In [46]:
s

,股票代码,股票名称,持有期限,买入日期,卖出日期,买入价格,卖出价格,收益率,浮盈,浮亏
0,sh600000,浦发银行,5,1999-11-12,1999-11-17,27.86,26.5,0.968984,1.0,0.946524


In [57]:
t2

,股票代码,股票名称,持有期限,胜率,平均收益,最大收益,最小收益,平均浮盈,最大浮盈,最小浮盈,平均浮亏,最大浮亏,最小浮亏
0,sh600000,浦发银行,5,0.452381,0.996992,1.168845,0.798088,1.029186,1.168845,1.0,0.959237,1.0,0.798088
